# Emma and Michael's Dank Boio Modeling and Simulation in Python (Copyright Allen B. Downey) Project Number 3 in the Eleventh Month of the Two Thousand and Eighteenth Year

In [11]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim library
from modsim import *
import math


In [12]:
def earth_radius(mass):
    density = 5514 # 5514 kg/m3 from NASA Earth Fact Sheet
    Re = (3*mass / (4*pi*density))**(1/3)
    return Re

def moon_radius(mass):
    density = 3344 # 3344 kg/m3 from NASA Moon Fact Sheet
    Rm = (3*mass / (4*pi*density))**(1/3)
    return Rm

In [13]:
params = Params(num_rockets = 2.5e8,
                r_e = earth_radius,
                r_m = moon_radius,
                G=6.674e-11, #N / kg**2 * m**2
                t_0=0,
                t_end=5*365*24*60*60, 
                x_0 = 362600000, # 362,600 km at perigee
                v_0 = 1078.2, # 1078.2 m/s at perigee
                Me_0 = 5.9722e24, # 5.9722×10^24 kg
                Mm_0 = 7.342e22); # 7.342×10^22 kg

In [14]:
def make_system(params):
    num_rockets, r_e, r_m, G, t_0, t_end, x_0, v_0, Me_0, Mm_0 = params
    
    dmdt = 6.54e6*num_rockets
    
    init = State(Me = Me_0, # Initial mass of earth
             Mm = Mm_0, #Initial mass of moon
             x_m=x_0, # 362,600 km at perigee
             y_m=0,
             vx_m=0,
             vy_m=v_0, # 1078.2 m/s at perigee
             x_e = 0,
             y_e = 0,
             vx_e = 0,
             vy_e = 0
            )
    system = System(init = init,
                dmdt =dmdt,
                r_e = earth_radius,
                r_m = moon_radius,
                G=6.674e-11, #N / kg**2 * m**2
                t_0=0,
                t_end=5*365*24*60*60, 
                x_0 = 362600000, # 362,600 km at perigee
                v_0 = 1078.2, # 1078.2 m/s at perigee
                Me_0 = 5.9722e24, # 5.9722×10^24 kg
                Mm_0 = 7.342e22) # 7.342×10^22 kg
    
    return system

system = make_system(params)

,values
init,Me 5.972200e+24 Mm 7.342000e+22 x_m ...
dmdt,1.635e+15
r_e,<function earth_radius at 0x000001BEFB9C1E18>
r_m,<function moon_radius at 0x000001BEFB9C1620>
G,6.674e-11
t_0,0
t_end,157680000
x_0,362600000
v_0,1078.2
Me_0,5.9722e+24


In [15]:
def net_force_moon(state, system):
    """Computes gravitational force.
    
    state: State object with distance r
    system: System object with m1, m2, and G
    """
    Me,Mm,x_m,y_m,vx_m,vy_m,x_e,y_e,vx_e,vy_e = state
    unpack(system)
    
    #Gravitational Force
    r = sqrt((x_m-x_e)**2 + (y_m-y_e)**2)
    force= G * Me * Mm / r**2
    direction = math.atan2(y_m-y_e,x_m-x_e) + pi 
    gravitational_force = Vector(force * math.cos(direction), force*math.sin(direction) )
    
    # Moon momentum contribution
    collision_force = dmdt * -(Vector(vx_m,vy_m))
    
    return gravitational_force + collision_force

In [16]:
def net_force_earth(state, system):
    """Computes gravitational force.
    
    state: State object with distance r
    system: System object with m1, m2, and G
    """
    Me,Mm,x_m,y_m,vx_m,vy_m,x_e,y_e,vx_e,vy_e = state
    unpack(system)
    
    #Gravitational Force
    r = sqrt((x_m-x_e)**2 + (y_m-y_e)**2)
    force= G * Me * Mm / r**2
    direction = math.atan2(y_m-y_e,x_m-x_e)
    gravitational_force = Vector(force * math.cos(direction), force*math.sin(direction) )
    
    return gravitational_force

In [17]:
def slope_func(state,t,system):
    
    Me,Mm,x_m,y_m,vx_m,vy_m,x_e,y_e,vx_e,vy_e = state
    unpack(system)
    V_m = Vector(vx_m,vy_m)
    V_e = Vector(vx_e,vy_e)
    
    Fnet_m = net_force_moon(state,system)
    Fnet_e = net_force_earth(state,system)
    
    dMedt = -dmdt
    dMmdt = dmdt
    dxdt_m = V_m
    dvdt_m = Fnet_m / Mm
    dxdt_e = V_e
    dvdt_e = Fnet_e / Me
    
    return dMedt, dMmdt, dxdt_m[0], dxdt_m[1], dvdt_m[0], dvdt_m[1], dxdt_e[0], dxdt_e[1], dvdt_e[0], dvdt_e[1],

In [18]:
def event_func(state,t,system):
    Me,Mm,x_m,y_m,vx_m,vy_m,x_e,y_e,vx_e,vy_e = state
    unpack(system)
   
    r = sqrt((x_m-x_e)**2 + (y_m-y_e)**2)
    collision_radius = earth_radius(Me) + moon_radius(Mm)
    
    #End the simulart if there is a collision OR if the orbital radius exceeds its initial value at apogee
    if(Me < 0):
        print('Negative mass is really sad...')
        return 0
    elif(r > 0.4055e10): # e9 originally
        print('The moon escaped our experiment!')
        return 0
    else:
        return r - collision_radius


In [19]:
def error_func(rockets, params):
    params = Params(params, num_rockets=rockets)
    system = make_system(params)
    results, details = run_ode_solver(system, slope_func, events=event_func, method="LSODA")
    results.index /= 24*60*60*365
    return results.index[-1] - 3.52

In [ ]:
optimum_rockets = fsolve(error_func,2.2e8,params)
print(optimum_rockets)

In [ ]:
results.index /= 24*60*60

In [ ]:
plot(results.x_m,results.y_m)
plot(results.x_e,results.y_e)

In [ ]:
results.index[-1]/365

In [ ]:
results.head()

In [ ]:
results.tail()